# Experiment Test with DeMiguel's Paper
Most Setup follows the same as before.


In [1]:
from main_head import *

## Overall Parametric Setup
The overall parameter choice is set in the ``CVaR_parameter.py``. Below are some of parameters selected in our experiment.

## Dataset Selection Part

In [2]:
#Data Set Choice Generator
def DTA_select(data_type):
    if data_type[-1] == '2':
        return [19970101, 201904, 20061231]
    else:
        if data_type in ['FF', 'IndustryPortfolios', 'MKT']:
            return [19630701, 200412, 19730631]
        elif data_type == 'S&P_sectors':
            return [19810101, 200212, 19901231]
        else:
            return [19700101, 200108, 19791231]
        
    
    return [start_time, end_time, train_test_split]

In [3]:
freq = "Monthly"
#Here as we all use the Monthly Data Set, we can fix that parameter
value = ""
#eq/not eq csv in the . Here we all use the original dataset class.

#dta type
## In DeMiguel's paper, the dta type are selected to be one in ['MKT', 'Country', 'S&P_sectors', 'IndustryPortfolios', 'FF']
## When his dataset extended to 2019 Q1, the dta type are selected to be one in ['MKT2', 'Country2', 'S&P_sectors2', 'IndustryPortfolios2', 'FF22']
## Their corresponding portfolio number is 3 (MKT), 9 (Country), 11 (S&P_sectors), 11 (IndustryPortfolios), 21 /24 for FF.
## And in our case study, dta type are selected to be 'IndustryIndices' (16)
data_type = "Country"
portfolio_number = 9

#select part of the data in the .csv file.
## from start_time to train_test_split as the default train dataset; from train_test_split (not include) as the test dataset.
[start_time, end_time, train_test_split] = DTA_select(data_type)

## Load the Data
we load the return data and its corresponding factor data below using the class from ``data_process/input.py``.

In [5]:
cv_type = 1
# default setup in CVaR_parameter.py to accept cross validation (with other parameters in CVaR_parameter.py)
# can check other values cv types for the other uses
#data input
Input_csv = Input(portfolio_number, freq, value, start_time, end_time, train_test_split, data_type)
[data_head, data_parameter, csv_name] = Input_csv.parameter_output()
[df_select, df_train] = Input_csv.data_load()

df_factor = Input_csv.three_factor_load()
#we do not include the risk free rate item into the computation of sharpe ratio
if sharpe_ratio_open == False:
    rfr_data = 0
else:
    rfr_data = Input_csv.risk_free_rate()

In [6]:
end_time

200108

# Our Proposed Model

### Pretrained for determining the Markovian States
We use the MATLAB hmmtrain function for training the HMM model. Here we only incorporate the HMM model for simplicity of interpretation.

HMM results are saved in ``factor model/HMM_state.csv``.

We would first load the HMM results within the right train test time.

In [9]:
cluster_num = 4
df_state = pd.read_csv("../factor model/HMM_state.csv")
str_state = [str(each_state) for each_state in list(df_state['Date'])]
df_state['Date'] = str_state
df_state_hmm = df_state[((df_state['Date'])>=str(start_time))&(df_state['Date']<str(end_time))]['state']
##hmm_type == -1 represents we use the pretrained result for our classification.
hmm_type = -1

In [10]:
cv_type

1

#### Our HMM Model: RSDR CVaR (HMM) 
Note that the expressions for the model are explicitly written in the paper.We follow the rolling window method as shown in another notebook and paper.

In [11]:
# RSDR CVaR (HMM)
method_name = "RSDR CVaR (HMM)"
mean_constr = False
fcvar_hmm_star = FCVaR_HMM_wasserstein(df_select, df_train, rolling_day, portfolio_number, df_factor, cluster_num, method_name, df_state_hmm, hmm_type, cv_type, mean_constr, ambiguity_param)
fcvar_hmm_star.rolling()

0
Using license file D:\ProgramFiles\gurobi903\gurobi.lic


GurobiError: License expired 2023-06-24

In [15]:
csv_name = f'../output/{portfolio_number}Monthly_{data_type}.csv'

The description of output overflow and performance metrics can be seen from ``portfolio_experiment_final_version.ipynb``.

In [16]:
#initiation for the test metric module
#print the output into a separate csv file, separated by the dataset type + header
Output_csv = output(csv_name, data_head, data_parameter)
Output_csv.head()
#return statistics computation for our policy
# Output_csv.return_info(fcvar_hmm_star, rfr_data)

done


## Regime Switching Portfolio
we set $\theta = 0$ and CV Type = -1.


In [172]:
method_name = "RS CVaR (HMM)"
mean_constr = False
fcvar_hmm_rs = FCVaR_HMM_wasserstein(df_select, df_train, rolling_day, portfolio_number, df_factor, cluster_num, method_name, df_state_hmm, hmm_type, -1, mean_constr, ambiguity_param)
fcvar_hmm_rs.theta = 0
fcvar_hmm_rs.rolling()
Output_csv.return_info(fcvar_hmm_rs, rfr_data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
For the model with strategy name  RS CVaR (HMM)
The Monthly Sharpe Ratio =

## Equally Weighted Portfolio
This equally weighted strategy means that we consider the portfolio weight to be as follows for each of the $I$ risky assets:
$$w_i = \frac{1}{I},$$
which gains popularity for its robustness in DeMiguel et al. (2009b).

In [17]:
# EW Portfolio
naive = naive_strategy(df_select, df_train, rolling_day, portfolio_number, "EW portfolio")
naive.rolling()
Output_csv.return_info(naive, rfr_data)

379 120
For the model with strategy name  EW portfolio
The Monthly Sharpe Ratio =  0.12766718860679807
CEQ =  0.004606591198094999
Drawdown =  -0.3940773786489601
Turnover =  0.03340784296101823


-0.3940773786489601

Change the $\textbf{dataset selection part}$ above to get all of the portfolio results.

# DR CVaR (Wasserstein)


In [174]:
#DR CVaR
mean_constr = False
cv_type = 1
#mean-FCVaR model (Wasserstein)
method_name = "DR CVaR (Wasserstein)"
mean_fcvar_wasserstein = FCVaR_wasserstein2(df_select, df_train, rolling_day, portfolio_number, 1, method_name, cv_type, mean_constr)
mean_fcvar_wasserstein.rolling()

In [175]:
Output_csv.return_info(mean_fcvar_wasserstein, rfr_data)

For the model with strategy name  DR CVaR (Wasserstein)
The Monthly Sharpe Ratio =  0.12152949533601597
CEQ =  0.004333712561030371
Drawdown =  -0.4065561119600003
Turnover =  0.054649473109679804
